In [1]:
import json
import importlib
import os
current_dir=os.getcwd()
os.chdir("../../")
# print(os.getcwd())
import dist_utilities as dist_ult
# import noise_est_utilities as ne_ult
importlib.reload(dist_ult)
# importlib.reload(ne_ult)
os.chdir(current_dir)
import pickle
from qiskit.result import Result
from qiskit.visualization import plot_histogram, plot_distribution
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy
from qiskit.result import QuasiDistribution

In [2]:
with open("ideal_counts_sim.json", "r") as f:
    ideal_counts=json.load(f)
with open("noisy_counts_sim.json", "r") as f:
    noisy_counts=json.load(f)
with open("noisy_noise_est_dist_sim.json", "r") as f:
    noisy_noise_est_dist=json.load(f)
with open("ideal_noise_est_counts_sim.json", "r") as f:
    ideal_noise_est_counts=json.load(f)
print("ideal: ", ideal_noise_est_counts)
print("noise est dist: ", noisy_noise_est_dist)
print("noisy counts: ", noisy_counts)
print("ideal counts: ", ideal_counts)
col_label=list(dict(ideal_noise_est_counts).keys())[0]
# corrected_counts=ne_ult.get_corrected_counts_from_counts(noisy_counts, noisy_noise_est_dist, list(ideal_noise_est_counts.keys())[0])

# print(corrected_counts)

ideal:  {'00000000010000000000': 200000}
noise est dist:  {'00010000010110000000': 1e-05, '00000001010011000000': 2e-05, '00000000010010000000': 0.0065, '00000000010000000000': 0.816315, '00000001010000101000': 5e-06, '00000010110000000000': 7.5e-05, '00000000010001000011': 5e-06, '11000000010000011100': 5e-06, '00000011010000000000': 0.002145, '00000000010000010000': 0.007105, '00000100010100000000': 5.5e-05, '00000000000000001100': 1e-05, '01000000011000000000': 5.5e-05, '00000000010010000110': 1.5e-05, '00000001110000000000': 0.00218, '00000100100000000100': 5e-06, '00000110011000000000': 1e-05, '00000000010010000010': 5.5e-05, '00000001110000000100': 2.5e-05, '00000000000000000110': 1e-05, '10000000011000001000': 5e-06, '00000000010000100000': 0.00678, '00000000000000000000': 0.00563, '00000000010000011100': 3.5e-05, '10000000010000110000': 1.5e-05, '00000000010100000000': 0.00659, '00000000110000000110': 1.5e-05, '00010000010000000000': 0.006665, '00000100010001000001': 5e-06, '00

In [3]:
# plot_histogram([ideal_counts, noisy_counts, corrected_counts], number_to_keep=5, legend=["ideal counts", "noisy counts", "corrected counts"], figsize=(20,20), filename="plot_sim.jpg")


In [4]:
num_qubits=len(list(noisy_counts.items())[0][0])

if 2**15<2**num_qubits:
    num_keep=2**10#num_qubits#2**10
else:
    num_keep=2**num_qubits
noisy_counts_orig=noisy_counts
print("num keep ", num_keep)
noisy_noise_est_dist=dist_ult.convert_to_first_col(noisy_noise_est_dist, col_label)
noisy_noise_est_dist=dist_ult.filter_counts(noisy_noise_est_dist, num_keep)
noisy_counts=dist_ult.filter_counts(noisy_counts, num_keep)
all_keys, noisy_noise_est_dist, noisy_counts=dist_ult.fill_counts_range(noisy_noise_est_dist, noisy_counts, num_keep, num_qubits)
print(noisy_counts)
print(noisy_noise_est_dist)

noisy_noise_est_vec=sorted(list(noisy_noise_est_dist.items()), key=lambda x: int(x[0], 2))
noisy_noise_est_vec=[elem[1] for elem in noisy_noise_est_vec]
counts_vec=sorted(list(noisy_counts.items()), key=lambda x: int(x[0], 2))
counts_vec=[elem[1] for elem in counts_vec]
# print(len(noisy_noise_est_vec))
# print(len(counts_vec))
quasi_corrected_counts=dist_ult.get_quasi_corrected(noisy_noise_est_vec, counts_vec)
quasi_corrected_counts=dist_ult.label_quasis(quasi_corrected_counts, all_keys)
# print(quasi_corrected_counts)
quasi_method=QuasiDistribution(quasi_corrected_counts)
quasi_corrected_counts=quasi_method.nearest_probability_distribution().binary_probabilities(num_qubits)
# print(quasi_corrected_counts)
# quasi_corrected_counts={bin(idx)[2::].zfill(num_qubits):val for idx, val in enumerate(quasi_corrected_counts)}



num keep  1024
max exist  1048575
min exist  0
size before  1623
{'00000000000000000000': 82025, '00000000000000000001': 464, '00000000000000000010': 242, '00000000000000000011': 603, '00000000000000000100': 250, '00000000000000000101': 4, '00000000000000000110': 4, '00000000000000000111': 627, '00000000000000001000': 251, '00000000000000001001': 1, '00000000000000001010': 1, '00000000000000001011': 4, '00000000000000001100': 6, '00000000000000001101': 1, '00000000000000001110': 0, '00000000000000001111': 650, '00000000000000010000': 256, '00000000000000010001': 4, '00000000000000010010': 0, '00000000000000010011': 0, '00000000000000010100': 2, '00000000000000010110': 0, '00000000000000010111': 5, '00000000000000011000': 9, '00000000000000011001': 0, '00000000000000011010': 0, '00000000000000011011': 1, '00000000000000011100': 4, '00000000000000011110': 4, '00000000000000011111': 659, '00000000000000100000': 259, '00000000000000100001': 0, '00000000000000100010': 1, '000000000000001000

In [5]:
fig=plot_histogram([ideal_counts, noisy_counts_orig, quasi_corrected_counts], number_to_keep=5, #bar_labels=False,
               legend=["ideal", "raw", "corrected"], title="GHZ 20 Qubits Simulation", bar_labels=False)
fig.savefig("plot_ghz20_counts_sim.jpg", bbox_inches="tight")

In [6]:
fig=plot_distribution([ideal_counts, noisy_counts_orig, quasi_corrected_counts], number_to_keep=5, #bar_labels=False,
                  legend=["ideal", "raw", "corrected"], title="GHZ 20 Qubits Simulation", bar_labels=False) #figsize=(15,15))
ax=fig.axes[0]
ax.set_ylabel("Probability")
fig.tight_layout()
fig.savefig("plot_ghz20_dist_sim.jpg", bbox_inches="tight")